[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Part_4/Task_4_RNN.ipynb)

# Домашнее задание 4
- Сравнить LSTM, RNN и GRU на задаче предсказания части речи (качество предсказания, скорость обучения, время инференса модели)
- *к первой задаче добавить bidirectional


In [8]:
import pandas as pd
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

### Загрузчик данных

In [9]:
# Формирование кастомного класса DatasetSeq на базе класса Dataset
class DatasetSeq(Dataset):
    #def __init__(self, data_dir, train_lang='en'):                 # при хранении файла на диске в colab
    def __init__(self, dataset_dir):
	#open file
        #with open(data_dir + train_lang + '.train', 'r') as f:     # при хранении файла на диске в colab
        with open(dataset_dir, 'r', encoding="utf-8") as f:         # датасет читает исходный файл
            train = f.read().split('\n\n')                          # и разбивает его на блока (абзацы)
                                                                    # добавлена кодировка - Python ругается

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        
        # формирование словарей токенов: таргет, слова и буквы (можно обрабатывать слова или буквы)     
        # паддинг выполняет роль как в свертках (привести последовательности к одной длине)
        self.target_vocab = {'<pad>': 0} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {'<pad>': 0} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {'<pad>': 0} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]
	    
        #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        # для каждого абзаца
        for line in train:
            # последовательность, таргет и символы - пустые списки
            sequence = []
            target = []
            chars = []
            # для каждой строки абзаца
            for item in line.split('\n'):
                # если строка не пуста
                if item != '':
                    # по пробелу разбиваем строку на слово и таргет
                    word, label = item.split(' ')
                    # если слова нет в словаре, оно добавляется в словарь с очередным номером (ключем/индексом)
                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    # если таргета нет в словаре, он добавляется в словарь с очередным номером (ключем/индексом)
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    # для каждого символа в слове
                    for char in word:
                        # если символа нет в словаре, он добавляется в словарь с очередным номером (ключем/индексом)
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    # к последовательности, таргету и символам добавляются слово, таргет и символы
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            # к словарям токенов добавляются полученные ранее последовательность, таргет и символы
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], # [1, 2, 3, 4, 6] len=5
        }

In [10]:
dataset_dir = 'en2.train'
dataset = DatasetSeq(dataset_dir)
dataset.__getitem__(6)

{'data': [67, 91, 92, 60, 61, 62, 15, 93, 21, 94, 95, 96, 26],
 'char': [[33, 22, 15],
  [25, 22, 4, 3, 20],
  [28, 2, 6],
  [23, 15, 4, 13, 34],
  [3, 5, 13],
  [23, 30],
  [25, 22, 15],
  [22, 15, 2, 20],
  [18, 17],
  [2, 13],
  [4, 13, 42, 15, 6, 25, 12, 15, 13, 25],
  [17, 4, 3, 12],
  [31]],
 'target': [6, 3, 8, 8, 5, 7, 6, 4, 7, 6, 4, 4, 2]}

### Формирование батча

In [11]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    # pad_sequence встроенная функция по дополнению батчей паддингом
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

### Гиперпараметры

In [12]:
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
emb_dim = 256
hidden = 256
n_epochs = 10
batch_size = 64               
cuda_device = 0
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'mps'

## Модель LSTM

### Архитектура модели

In [13]:
class LSTMPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в LSTM
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
    
    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в LSTM
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [14]:
LSTM_model = LSTMPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
LSTM_model.train()
optim = torch.optim.Adam(LSTM_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [15]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = LSTM_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
LSTM_train_time = datetime.datetime.now() - start
LSTM_train_loss = loss.item()
print(f"Время обучения модели LSTM: {LSTM_train_time}")
print(f"Итоговый loss LSTM на обучающей выборке: {LSTM_train_loss}")

epoch: 0, step: 0, loss: 2.986635446548462
epoch: 0, step: 100, loss: 0.40807098150253296
epoch: 0, step: 200, loss: 0.23439031839370728
epoch: 0, step: 300, loss: 0.15080615878105164
epoch: 1, step: 0, loss: 0.15749505162239075
epoch: 1, step: 100, loss: 0.13533201813697815
epoch: 1, step: 200, loss: 0.13321822881698608
epoch: 1, step: 300, loss: 0.09757743775844574
epoch: 2, step: 0, loss: 0.10443088412284851
epoch: 2, step: 100, loss: 0.07949259877204895
epoch: 2, step: 200, loss: 0.10325685143470764
epoch: 2, step: 300, loss: 0.10741005837917328
epoch: 3, step: 0, loss: 0.0762549415230751
epoch: 3, step: 100, loss: 0.10484195500612259
epoch: 3, step: 200, loss: 0.05027348920702934
epoch: 3, step: 300, loss: 0.05924995243549347
epoch: 4, step: 0, loss: 0.05131750926375389
epoch: 4, step: 100, loss: 0.051714491099119186
epoch: 4, step: 200, loss: 0.05728011205792427
epoch: 4, step: 300, loss: 0.05422991141676903
epoch: 5, step: 0, loss: 0.05132881924510002
epoch: 5, step: 100, loss: 

### Инференс

In [16]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    LSTM_model.eval()
    predict = LSTM_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    LSTM_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели LSTM: {LSTM_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели LSTM: 0:00:00.004986


## Модель RNN

### Архитектура модели

In [17]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [18]:
RNN_model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
RNN_model.train()
optim = torch.optim.Adam(RNN_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [19]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = RNN_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
RNN_train_time = datetime.datetime.now() - start
RNN_train_loss = loss.item()
print(f"Время обучения модели RNN: {RNN_train_time}")
print(f"Итоговый loss RNN на обучающей выборке: {RNN_train_loss}")

epoch: 0, step: 0, loss: 2.875800609588623
epoch: 0, step: 100, loss: 0.3565364181995392
epoch: 0, step: 200, loss: 0.25201037526130676
epoch: 0, step: 300, loss: 0.09936951100826263
epoch: 1, step: 0, loss: 0.17948633432388306
epoch: 1, step: 100, loss: 0.21085622906684875
epoch: 1, step: 200, loss: 0.15002746880054474
epoch: 1, step: 300, loss: 0.10990066081285477
epoch: 2, step: 0, loss: 0.12464043498039246
epoch: 2, step: 100, loss: 0.0723038911819458
epoch: 2, step: 200, loss: 0.10167223960161209
epoch: 2, step: 300, loss: 0.09535925835371017
epoch: 3, step: 0, loss: 0.11156997829675674
epoch: 3, step: 100, loss: 0.08491683006286621
epoch: 3, step: 200, loss: 0.1310947835445404
epoch: 3, step: 300, loss: 0.09874419867992401
epoch: 4, step: 0, loss: 0.07996009290218353
epoch: 4, step: 100, loss: 0.08710276335477829
epoch: 4, step: 200, loss: 0.07596481591463089
epoch: 4, step: 300, loss: 0.043718572705984116
epoch: 5, step: 0, loss: 0.05649275705218315
epoch: 5, step: 100, loss: 0.

### Инференс

In [20]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    RNN_model.eval()
    predict = RNN_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    RNN_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {RNN_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели RNN: 0:00:00.003990


## Модель GRU

### Архитектура модели

In [21]:
class GRUPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [22]:
GRU_model = GRUPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
GRU_model.train()
optim = torch.optim.Adam(GRU_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [23]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = GRU_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
GRU_train_time = datetime.datetime.now() - start
GRU_train_loss = loss.item()
print(f"Время обучения модели GRU: {GRU_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {GRU_train_loss}")

epoch: 0, step: 0, loss: 2.925163745880127
epoch: 0, step: 100, loss: 0.39371463656425476
epoch: 0, step: 200, loss: 0.2202056348323822
epoch: 0, step: 300, loss: 0.22839149832725525
epoch: 1, step: 0, loss: 0.19430921971797943
epoch: 1, step: 100, loss: 0.16816985607147217
epoch: 1, step: 200, loss: 0.15221503376960754
epoch: 1, step: 300, loss: 0.12427293509244919
epoch: 2, step: 0, loss: 0.041089266538619995
epoch: 2, step: 100, loss: 0.12064767628908157
epoch: 2, step: 200, loss: 0.09291797131299973
epoch: 2, step: 300, loss: 0.07890412956476212
epoch: 3, step: 0, loss: 0.0991993248462677
epoch: 3, step: 100, loss: 0.055127453058958054
epoch: 3, step: 200, loss: 0.08650507032871246
epoch: 3, step: 300, loss: 0.09460203349590302
epoch: 4, step: 0, loss: 0.053088631480932236
epoch: 4, step: 100, loss: 0.07211247086524963
epoch: 4, step: 200, loss: 0.0578676201403141
epoch: 4, step: 300, loss: 0.058903325349092484
epoch: 5, step: 0, loss: 0.06764907389879227
epoch: 5, step: 100, loss:

### Инференс

In [24]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    GRU_model.eval()
    predict = GRU_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    GRU_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели GRU: {GRU_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели GRU: 0:00:00.001995


## Модель Bidirectional LSTM

### Архитектура модели

In [25]:
class BidirLSTMPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [26]:
BidirLSTM_model = BidirLSTMPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirLSTM_model.train()
optim = torch.optim.Adam(BidirLSTM_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [27]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirLSTM_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirLSTM_train_time = datetime.datetime.now() - start
BidirLSTM_train_loss = loss.item()
print(f"Время обучения модели LSTM: {BidirLSTM_train_time}")
print(f"Итоговый loss LSTM на обучающей выборке: {BidirLSTM_train_loss}")

epoch: 0, step: 0, loss: 2.974355936050415
epoch: 0, step: 100, loss: 0.28779786825180054
epoch: 0, step: 200, loss: 0.23163756728172302
epoch: 0, step: 300, loss: 0.15558332204818726
epoch: 1, step: 0, loss: 0.09718077629804611
epoch: 1, step: 100, loss: 0.14829786121845245
epoch: 1, step: 200, loss: 0.11599984019994736
epoch: 1, step: 300, loss: 0.07546371221542358
epoch: 2, step: 0, loss: 0.07108955830335617
epoch: 2, step: 100, loss: 0.07560165971517563
epoch: 2, step: 200, loss: 0.07052402198314667
epoch: 2, step: 300, loss: 0.0738890990614891
epoch: 3, step: 0, loss: 0.048259954899549484
epoch: 3, step: 100, loss: 0.04757946729660034
epoch: 3, step: 200, loss: 0.053839653730392456
epoch: 3, step: 300, loss: 0.06078903004527092
epoch: 4, step: 0, loss: 0.030623503029346466
epoch: 4, step: 100, loss: 0.0337982140481472
epoch: 4, step: 200, loss: 0.03134100139141083
epoch: 4, step: 300, loss: 0.02424294501543045
epoch: 5, step: 0, loss: 0.025052865967154503
epoch: 5, step: 100, loss

### Инференс

In [28]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirLSTM_model.eval()
    predict = BidirLSTM_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirLSTM_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели LSTM: {BidirLSTM_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели RNN: 0:00:00.005983


## Модель Bidirectional RNN

### Архитектура модели

In [29]:
class BidirRNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [30]:
BidirRNN_model = BidirRNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirRNN_model.train()
optim = torch.optim.Adam(BidirRNN_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [31]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirRNN_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirRNN_train_time = datetime.datetime.now() - start
BidirRNN_train_loss = loss.item()
print(f"Время обучения модели RNN: {BidirRNN_train_time}")
print(f"Итоговый loss RNN на обучающей выборке: {BidirRNN_train_loss}")

epoch: 0, step: 0, loss: 3.1870741844177246
epoch: 0, step: 100, loss: 0.25928953289985657
epoch: 0, step: 200, loss: 0.18400955200195312
epoch: 0, step: 300, loss: 0.1998547911643982
epoch: 1, step: 0, loss: 0.1704816222190857
epoch: 1, step: 100, loss: 0.10727096349000931
epoch: 1, step: 200, loss: 0.1522814929485321
epoch: 1, step: 300, loss: 0.10622724145650864
epoch: 2, step: 0, loss: 0.11479531228542328
epoch: 2, step: 100, loss: 0.08860860019922256
epoch: 2, step: 200, loss: 0.09459187835454941
epoch: 2, step: 300, loss: 0.07238802313804626
epoch: 3, step: 0, loss: 0.05111191049218178
epoch: 3, step: 100, loss: 0.058269061148166656
epoch: 3, step: 200, loss: 0.0430125892162323
epoch: 3, step: 300, loss: 0.08022809028625488
epoch: 4, step: 0, loss: 0.05248823016881943
epoch: 4, step: 100, loss: 0.030445965006947517
epoch: 4, step: 200, loss: 0.03769374638795853
epoch: 4, step: 300, loss: 0.06707563251256943
epoch: 5, step: 0, loss: 0.03551698848605156
epoch: 5, step: 100, loss: 0

In [32]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirRNN_model.eval()
    predict = BidirRNN_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirRNN_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {BidirRNN_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADV', 'ADJ', 'PRON', 'VERB', 'PRON']
Время инференса модели RNN: 0:00:00.000998


## Модель Bidirectional GRU

### Архитектура модели

In [33]:
class BidirGRUPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [34]:
BidirGRU_model = BidirGRUPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirGRU_model.train()
optim = torch.optim.Adam(BidirGRU_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [35]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirGRU_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirGRU_train_time = datetime.datetime.now() - start
BidirGRU_train_loss = loss.item()
print(f"Время обучения модели GRU: {BidirGRU_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {BidirGRU_train_loss}")

epoch: 0, step: 0, loss: 3.2253525257110596
epoch: 0, step: 100, loss: 0.16711539030075073
epoch: 0, step: 200, loss: 0.15875305235385895
epoch: 0, step: 300, loss: 0.19771325588226318
epoch: 1, step: 0, loss: 0.11242194473743439
epoch: 1, step: 100, loss: 0.11645243316888809
epoch: 1, step: 200, loss: 0.09003214538097382
epoch: 1, step: 300, loss: 0.11728061735630035
epoch: 2, step: 0, loss: 0.09017952531576157
epoch: 2, step: 100, loss: 0.07458610832691193
epoch: 2, step: 200, loss: 0.08346608281135559
epoch: 2, step: 300, loss: 0.08584064990282059
epoch: 3, step: 0, loss: 0.046209435909986496
epoch: 3, step: 100, loss: 0.0766185075044632
epoch: 3, step: 200, loss: 0.040786970406770706
epoch: 3, step: 300, loss: 0.043938398361206055
epoch: 4, step: 0, loss: 0.02935095503926277
epoch: 4, step: 100, loss: 0.017926497384905815
epoch: 4, step: 200, loss: 0.03432294726371765
epoch: 4, step: 300, loss: 0.03164127841591835
epoch: 5, step: 0, loss: 0.02312261052429676
epoch: 5, step: 100, lo

### Инференс

In [36]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirGRU_model.eval()
    predict = BidirGRU_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirGRU_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {BidirGRU_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADV', 'ADJ', 'PRON', 'VERB', 'PRON']
Время инференса модели RNN: 0:00:00.001995


## Сведем результаты в таблицу

In [37]:
result = pd.DataFrame(data=[[LSTM_train_time, LSTM_train_loss, LSTM_inference_time],
                            [BidirLSTM_train_time, BidirLSTM_train_loss, BidirLSTM_inference_time],
                            [RNN_train_time, RNN_train_loss, RNN_inference_time],
                            [BidirRNN_train_time, BidirRNN_train_loss, BidirRNN_inference_time],
                            [GRU_train_time, GRU_train_loss, GRU_inference_time],
                            [BidirGRU_train_time, BidirGRU_train_loss, BidirGRU_inference_time]],
                      index = ['LSTM','Bidirectional LSTM','RNN','Bidirectional RNN','GRU','Bidirectional GRU'],
                      columns = ['Время обучения', 'Loss на обучающей выборке', 'Время инференса'])
result

,Время обучения,Loss на обучающей выборке,Время инференса
LSTM,0 days 00:03:09.865320,0.031769,0 days 00:00:00.004986
Bidirectional LSTM,0 days 00:05:12.256995,0.005682,0 days 00:00:00.005983
RNN,0 days 00:02:41.726528,0.041238,0 days 00:00:00.003990
Bidirectional RNN,0 days 00:03:04.080257,0.017160,0 days 00:00:00.000998
GRU,0 days 00:03:10.953375,0.027206,0 days 00:00:00.001995
Bidirectional GRU,0 days 00:04:11.358445,0.003421,0 days 00:00:00.001995


Выводы:
- уменьшение размера батча уменьшило loss (для обучаеющей выборки), но увеличило время обучения;
- bidirectional удваивает время обучения (что логично), но loss на обучающей выборке уменьшается колоссально